Install necessary libraries `scipy` for scientific computing and `seaborn` for data visualization capabilities.Optuna for hyperparameter tuning

In [ ]:
# Install Required Libraries
!pip install scipy seaborn

Mount Google Drive to access the data files stored there

In [ ]:
# Mount Google Drive to access data files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import the required libraries. `numpy` for numerical operations, `scipy.io` for loading MATLAB files, `sklearn` for machine learning model creation, training, and evaluation, `matplotlib.pyplot` and `seaborn` for data visualization. Other libraries for ML architectures and for evaluating them

In [ ]:
# Import necessary libraries
import numpy as np
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

Load the coherence data from a MATLAB (.mat) file stored in Google Drive. It contains the coherence matrices for Alzheimer's Disease (AZ), Frontotemporal Dementia (FTD), and Healthy Controls (HC). Use `scipy.io.loadmat` to read the .mat file

In [ ]:
# Load the coherence data from .mat file
mat_file_path = '/content/drive/MyDrive/GNN_Approach/Data_files/Coherence.mat'
mat = scipy.io.loadmat(mat_file_path)
coherence_data = mat['Coherence'][0, 0]

In [ ]:
import scipy.io
import numpy as np

# Load the .mat file
file_path = '/content/drive/MyDrive/GNN_Approach/Subjects/Subject1.mat'
mat_data = scipy.io.loadmat(file_path)

# Extract the TF matrix
TF = mat_data['TF']

# Define the submatrices as specified
submatrix1 = TF[:, :, 0:8]
submatrix2 = TF[:, :, 8:15]
submatrix3 = TF[:, :, 15:27]
submatrix4 = TF[:, :, 27:60]
submatrix5 = TF[:, :, 60:92]

# Compute the average along the third dimension for each submatrix
averageMatrix1 = np.mean(submatrix1, axis=2)
averageMatrix2 = np.mean(submatrix2, axis=2)
averageMatrix3 = np.mean(submatrix3, axis=2)
averageMatrix4 = np.mean(submatrix4, axis=2)
averageMatrix5 = np.mean(submatrix5, axis=2)

# Combine the averages into a new matrix with the shape 2346*1*5
averageMatrix_combined = np.stack([averageMatrix1, averageMatrix2, averageMatrix3, averageMatrix4, averageMatrix5], axis=2)

# Reshape the matrix from 2346*1*5 to 2346*5*1
averageMatrix_reshaped = np.transpose(averageMatrix_combined, (0, 2, 1))

# Now, averageMatrix_reshaped has the shape 2346*5*1
# print(averageMatrix_reshaped.shape)

Patient_matrix = averageMatrix_reshaped


Extract the Coherence data for different categories: Alzheimer's Disease (AZ), Frontotemporal Dementia (FTD), and Healthy Controls (HC)

In [ ]:
# Extracting data in different categories (AZ, FTD, HC)
coherence_AZ_F = coherence_data['AZ']
coherence_FTD_F = coherence_data['FTD']
coherence_HC_F = coherence_data['HC']


Extract the data corresponding to the 3rd frequency band (index 2) and then reshape the extracted data

In [ ]:
# Extract the 3rd frequency band (index 2) and normalize the data
coherence_AZ = coherence_AZ_F[:, : , :].reshape(2346, 5, 36)
coherence_FTD = coherence_FTD_F[:, : , :].reshape(2346, 5, 23)
coherence_HC = coherence_HC_F[:, : , :].reshape(2346, 5, 29)

# Optional: To test whether the data is converted properly into numpy arrays
# print(coherence_AZ.shape)
# print(coherence_FTD.shape)
# print(coherence_HC.shape)
# print(Patient_matrix.shape)

Create Feature mask with given P values for 3 Categories. Here you can give the P values to select the features which are used to masking the connectivity data

In [ ]:

# Reshape data to (2346, 5, 36), (2346, 5, 29), (2346, 5, 23)
fm_data_az = coherence_AZ.reshape(2346, 5, 36)
fm_data_hc = coherence_HC.reshape(2346, 5, 29)
fm_data_ftd = coherence_FTD.reshape(2346, 5, 23)

alpha_az_hc = 0.01
alpha_ftd_hc = 0.01
alpha_az_ftd = 0.05

p_values_az_hc = np.zeros((fm_data_az.shape[0], fm_data_az.shape[1]))
p_values_ftd_hc = np.zeros((fm_data_ftd.shape[0], fm_data_ftd.shape[1]))
p_values_az_ftd = np.zeros((fm_data_az.shape[0], fm_data_az.shape[1]))

# Calculate p-values for AZ vs HC
for i in range(fm_data_az.shape[0]):
    for j in range(fm_data_az.shape[1]):
        _, p_value = ttest_ind(fm_data_az[i, j, :], fm_data_hc[i, j, :], equal_var=False) # welch's t test (Varient of Independent two sample t test)
        p_values_az_hc[i, j] = p_value

# Calculate p-values for FTD vs HC
for i in range(fm_data_ftd.shape[0]):
    for j in range(fm_data_ftd.shape[1]):
        _, p_value = ttest_ind(fm_data_ftd[i, j, :], fm_data_hc[i, j, :], equal_var=False)
        p_values_ftd_hc[i, j] = p_value

# Calculate p-values for AZ vs FTD
for i in range(fm_data_az.shape[0]):
    for j in range(fm_data_az.shape[1]):
        _, p_value = ttest_ind(fm_data_az[i, j, :], fm_data_ftd[i, j, :], equal_var=False)
        p_values_az_ftd[i, j] = p_value

# Create boolean masks
mask_az_hc = (p_values_az_hc < alpha_az_hc).astype(int).reshape(2346, 5, 1)
mask_ftd_hc = (p_values_ftd_hc < alpha_ftd_hc).astype(int).reshape(2346, 5, 1)
mask_az_ftd = (p_values_az_ftd < alpha_az_ftd).astype(int).reshape(2346, 5, 1)

# print("Shapes of masks:")
# print(mask_az_hc.shape)
# print(mask_ftd_hc.shape)
# print(mask_az_ftd.shape)

def calculate_significant_percentage(feature_mask):
    total_features = feature_mask.size
    significant_features = np.sum(feature_mask)
    percentage = (significant_features / total_features) * 100
    return percentage


# Calculate and print percentage of significant features for individual masks
# percentage_significant_AZ_HC = calculate_significant_percentage(mask_az_hc)
# print("AZ_HC Feature mask):")
# print(mask_az_hc.shape)
# print(f"Percentage of significant features AZ vs HC : {percentage_significant_AZ_HC:.2f}%")

# percentage_significant_FTD_HC = calculate_significant_percentage(mask_ftd_hc)
# print("FTD_HC Feature mask):")
# print(mask_ftd_hc.shape)
# print(f"Percentage of significant features FTD vs HC : {percentage_significant_FTD_HC:.2f}%")

# percentage_significant_AZ_FTD = calculate_significant_percentage(mask_az_ftd)
# print("AZ_FTD Feature mask):")
# print(mask_az_ftd.shape)
# print(f"Percentage of significant features AZ vs FTD  : {percentage_significant_AZ_FTD:.2f}%")

# Create combined feature mask
combined_feature_mask = ((mask_az_hc + mask_ftd_hc + mask_az_ftd) > 0).astype(int)

# Calculate and print percentage of significant features for combined mask
# percentage_significant_combined = calculate_significant_percentage(combined_feature_mask)
# print("Combined Feature mask:")
# print(combined_feature_mask.shape)
# print(f"Percentage of significant features in combined mask: {percentage_significant_combined:.2f}%")


/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


Shapes of masks:
(2346, 5, 1)
(2346, 5, 1)
(2346, 5, 1)
AZ_HC Feature mask):
(2346, 5, 1)
Percentage of significant features AZ vs HC : 11.24%
FTD_HC Feature mask):
(2346, 5, 1)
Percentage of significant features FTD vs HC : 4.68%
AZ_FTD Feature mask):
(2346, 5, 1)
Percentage of significant features AZ vs FTD  : 4.50%
Combined Feature mask:
(2346, 5, 1)
Percentage of significant features in combined mask: 16.05%


Band Filter for the above Features obtained with different P values. set the bands you dont want and make them into zero

In [ ]:
bands_to_zero = []

mask_az_hc[:, bands_to_zero, :] = 0
mask_ftd_hc[:, bands_to_zero, :] = 0
mask_az_ftd[:, bands_to_zero, :] = 0

# Create combined feature mask
combined_feature_mask = ((mask_az_hc + mask_ftd_hc + mask_az_ftd) > 0).astype(int)

# # Calculate and print percentage of significant features for individual masks
# percentage_significant_AZ_HC = calculate_significant_percentage(mask_az_hc)
# print("AZ_HC Feature mask):")
# print(mask_az_hc.shape)
# print(f"Percentage of significant features AZ vs HC : {percentage_significant_AZ_HC:.2f}%")

# percentage_significant_FTD_HC = calculate_significant_percentage(mask_ftd_hc)
# print("FTD_HC Feature mask):")
# print(mask_ftd_hc.shape)
# print(f"Percentage of significant features FTD vs HC : {percentage_significant_FTD_HC:.2f}%")

# percentage_significant_AZ_FTD = calculate_significant_percentage(mask_az_ftd)
# print("AZ_FTD Feature mask):")
# print(mask_az_ftd.shape)
# print(f"Percentage of significant features AZ vs FTD  : {percentage_significant_AZ_FTD:.2f}%")

# # Calculate and print percentage of significant features for combined mask
# percentage_significant_combined = calculate_significant_percentage(combined_feature_mask)
# print("Combined Feature mask:")
# print(combined_feature_mask.shape)
# print(f"Percentage of significant features in combined mask: {percentage_significant_combined:.2f}%")

AZ_HC Feature mask):
(2346, 5, 1)
Percentage of significant features AZ vs HC : 11.24%
FTD_HC Feature mask):
(2346, 5, 1)
Percentage of significant features FTD vs HC : 4.68%
AZ_FTD Feature mask):
(2346, 5, 1)
Percentage of significant features AZ vs FTD  : 4.50%
Combined Feature mask:
(2346, 5, 1)
Percentage of significant features in combined mask: 16.05%


Reconstructs the coherence matrix from the extracted coherence data. The function iterates over frequency bands and subjects, creating a 68x68 matrix with coherence values. The rebuilt matrices are stored in a 4D array with dimensions corresponding to regions, regions, bands, and subjects

In [ ]:
# Function to rebuild the coherence matrix from the coherence data
def rebuild_matrix(coherence_data):
    n_regions = 68
    n_data = coherence_data.shape[2]
    n_bands = coherence_data.shape[1]

    rebuilt_data = np.zeros((n_regions, n_regions, n_bands, n_data))

    for band in range(n_bands):
        for subject in range(n_data):
            matrix = np.zeros((n_regions, n_regions))
            idx = 0
            for col in range(n_regions):
                for row in range(col + 1):
                    matrix[row, col] = coherence_data[idx, band, subject]
                    idx += 1
            matrix = matrix + matrix.T - np.diag(matrix.diagonal())
            rebuilt_data[:, :, band, subject] = matrix

    return rebuilt_data

Reconstruct Mask Matrix

In [ ]:
# Function to rebuild the Feature mask matrix from mask data

def rebuild_matrix_mask(mask_data):
    n_regions = 68
    # n_data = coherence_data.shape[2]
    n_data = 1
    n_bands = mask_data.shape[1]

    rebuilt_data = np.zeros((n_regions, n_regions, n_bands, n_data))

    for band in range(n_bands):
        for subject in range(n_data):
            matrix = np.zeros((n_regions, n_regions))
            idx = 0
            for col in range(n_regions):
                for row in range(col + 1):
                    matrix[row, col] = mask_data[idx, band, subject]
                    idx += 1
            matrix = matrix + matrix.T - np.diag(matrix.diagonal())
            rebuilt_data[:, :, band, subject] = matrix

    return rebuilt_data

Reconstruct the coherence matrices for each category (AZ, FTD, HC) and check the shapes of the resulting matrices to ensure that they have been reconstructed correctly and have the expected dimensions

In [ ]:
# Rebuild the coherence matrices
coherence_AZ_rebuilt = rebuild_matrix(coherence_AZ)
coherence_FTD_rebuilt = rebuild_matrix(coherence_FTD)
coherence_HC_rebuilt = rebuild_matrix(coherence_HC)
Patient_rebuilt = rebuild_matrix(Patient_matrix)

# Check the shapes to ensure correctness
# print('COH_AZ_rebuilt shape:', coherence_AZ_rebuilt.shape)
# print('COH_FTD_rebuilt shape:', coherence_FTD_rebuilt.shape)
# print('COH_HC_rebuilt shape:', coherence_HC_rebuilt.shape)
# print('Patient_rebuilt shape:', Patient_rebuilt.shape)

Mask

In [ ]:
# Rebuild the coherence matrices
combined_mask_rebuilt = rebuild_matrix_mask(combined_feature_mask)

# Check the shapes to ensure correctness
# print('combined_mask_rebuilt shape:', combined_mask_rebuilt.shape)

count_combined_mask_rebuilt  = np.sum(combined_mask_rebuilt)
# print("Number of True values in combined mask:", count_combined_mask_rebuilt)

# print(combined_mask_rebuilt[:,:,2,0])

Extracts edge features from the coherence matrices and assigns labels to them. The function iterates over the subjects and bands, extracting the upper triangular part of each coherence matrix (excluding the diagonal). These edge attributes are then stored in a feature array, and the corresponding labels are stored in a label array

In [ ]:
def extract_edge_features_labels(data, label):
    features = []
    labels = []

    # Iterate over each sample
    for i in range(data.shape[3]):  # Samples dimension (36)
        combined_edge_attr = []

        # Iterate over each frequency band
        for j in range(data.shape[2]):  # Frequency bands dimension (5)
            matrix = data[:, :, j, i]  # Extract the 68x68 matrix for the j-th frequency band and i-th sample
            edge_attr = matrix[np.triu_indices_from(matrix, k=1)]  # Extract upper triangular part (excluding diagonal)
            combined_edge_attr.extend(edge_attr)  # Combine features from all frequency bands

        features.append(combined_edge_attr)  # Append combined features for the current sample
        labels.append(label)  # Append the label for the current sample

    return np.array(features), np.array(labels)

Use the `extract_edge_features_labels` function to extract features and labels for different classification tasks:
1. Alzheimer's Disease (AZ) vs. Healthy Controls (HC)
2. Frontotemporal Dementia (FTD) vs. Healthy Controls (HC)
3. Frontotemporal Dementia (FTD) vs. Alzheimer's Disease (AZ)

For each classification task, extract the edge features and labels from the rebuilt coherence matrices and concatenate them to form the complete dataset for each classification task

In [ ]:
# Extract features and labels for AZ vs HC
features_az_hc, labels_az_hc = extract_edge_features_labels(coherence_HC_rebuilt, 0)
features_az_hc_, labels_az_hc_ = extract_edge_features_labels(coherence_AZ_rebuilt, 2)
features_az_hc = np.concatenate((features_az_hc, features_az_hc_), axis=0)
labels_az_hc = np.concatenate((labels_az_hc, labels_az_hc_), axis=0)

# Extract features and labels for FTD vs HC
features_ftd_hc, labels_ftd_hc = extract_edge_features_labels(coherence_HC_rebuilt, 0)
features_ftd_hc_, labels_ftd_hc_ = extract_edge_features_labels(coherence_FTD_rebuilt, 1)
features_ftd_hc = np.concatenate((features_ftd_hc, features_ftd_hc_), axis=0)
labels_ftd_hc = np.concatenate((labels_ftd_hc, labels_ftd_hc_), axis=0)

# Extract features and labels for FTD vs AZ
features_ftd_az, labels_ftd_az = extract_edge_features_labels(coherence_FTD_rebuilt, 1)
features_ftd_az_, labels_ftd_az_ = extract_edge_features_labels(coherence_AZ_rebuilt, 2)
features_ftd_az = np.concatenate((features_ftd_az, features_ftd_az_), axis=0)
labels_ftd_az = np.concatenate((labels_ftd_az, labels_ftd_az_), axis=0)



Mask

In [ ]:
# Extract mask features and mask labels
combined_features_masks, labels_mask_az_hc = extract_edge_features_labels(combined_mask_rebuilt, 7)
combined_features_masks = combined_features_masks.astype(bool)




Patient

In [ ]:
features_patient,label_patient = extract_edge_features_labels(Patient_rebuilt, 8)

Feature summary

In [ ]:
# print(features_az_hc)
# print(features_az_hc.shape)
# print(features_ftd_hc)
# print(features_ftd_hc.shape)
# print(features_ftd_az)
# print(features_ftd_az.shape)
# print(features_patient)
# print(features_patient.shape)

# print(combined_features_masks)
# print(combined_features_masks.shape)


# Count the number of True values
count_combined_mask = np.sum(combined_features_masks)
print("Number of True values in combined mask:", count_combined_mask)


Number of True values in combined mask: 1883


Feature masking and selecting

In [ ]:

combined_feature_mask_flat = combined_features_masks.flatten()
# print(combined_feature_mask_flat)
# print(combined_feature_mask_flat.shape)

# Apply the mask to extract only the selected features
masked_features_az_hc = features_az_hc[:, combined_feature_mask_flat]
masked_features_ftd_hc = features_ftd_hc[:, combined_feature_mask_flat]
masked_features_ftd_az = features_ftd_az[:, combined_feature_mask_flat]
masked_features_patient = features_patient[:, combined_feature_mask_flat]

# print("Original AZ_HC features shape:", features_az_hc.shape)
# print("Mask AZ_HC shape:", combined_feature_mask_flat.shape)
# print("Masked AZ_HC features shape:", masked_features_az_hc.shape)

# print("Original FTD_HC features shape:", features_ftd_hc.shape)
# print("Mask FTD_HC shape:", combined_feature_mask_flat.shape)
# print("Masked FTD_HC features shape:", masked_features_ftd_hc.shape)

# print("Original FTD_AZ features shape:", features_ftd_az.shape)
# print("Mask FTD_AZ shape:", combined_feature_mask_flat.shape)
# print("Masked FTD_AZ features shape:", masked_features_ftd_az.shape)

# print("Original Patient features shape:", features_patient.shape)
# print("Mask Patient shape:", combined_feature_mask_flat.shape)
# print("Masked Patient features shape:", masked_features_patient.shape)


Visualize the edge features of a coherence matrix. The function takes the features, label, and title as inputs and plots the edge features using Matplotlib anlog with the labels

Split the extracted features and labels into training and testing sets for three classification tasks:
1. Alzheimer's Disease (AZ) vs. Healthy Controls (HC)
2. Frontotemporal Dementia (FTD) vs. Healthy Controls (HC)
3. Frontotemporal Dementia (FTD) vs. Alzheimer's Disease (AZ)

In [ ]:
# Split data into training and testing sets for AZ vs HC
X_train_az_hc, X_test_az_hc, y_train_az_hc, y_test_az_hc = train_test_split(masked_features_az_hc, labels_az_hc, test_size=0.2, random_state=42)

# Split data into training and testing sets for FTD vs HC
X_train_ftd_hc, X_test_ftd_hc, y_train_ftd_hc, y_test_ftd_hc = train_test_split(masked_features_ftd_hc, labels_ftd_hc, test_size=0.2, random_state=42)

# Split data into training and testing sets for FTD vs AZ
X_train_ftd_az, X_test_ftd_az, y_train_ftd_az, y_test_ftd_az = train_test_split(masked_features_ftd_az, labels_ftd_az, test_size=0.2, random_state=42)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Define the SVC models with the provided parameters

# svc_az_hc = SVC(C=2.1214906873874253, gamma=0.004484292508881285, kernel='rbf', probability=True)
# {'C': 0.11270661472823854, 'gamma': 0.16554604989987254, 'kernel': 'poly'}
# {'C': 0.08417824373866711, 'kernel': 'linear'}
svc_az_hc = SVC(C=0.08417824373866711, kernel='linear', probability=True)


# svc_ftd_hc = SVC(C=4.052602921806768, gamma=0.0042184837966682395, kernel='rbf', probability=True)
# {'C': 0.03208690181351073, 'gamma': 0.18157193875483335, 'kernel': 'linear'}
#  {'C': 1.6953850458743818, 'kernel': 'rbf', 'gamma': 'scale'}
svc_ftd_hc = SVC(C= 1.6953850458743818, gamma='scale', kernel='rbf', probability=True)


# svc_ftd_az = SVC(C=218.092808835883, kernel='sigmoid', gamma='scale', probability=True)

# {'C': 233.74681216558378, 'kernel': 'sigmoid', 'gamma': 'scale'}
# {'C': 256.2063292041088, 'gamma': 0.01425320509380262, 'kernel': 'sigmoid'}
# {'C': 264.97543383155426, 'kernel': 'sigmoid', 'gamma': 'scale'}
svc_ftd_az = SVC(C=256.2063292041088, kernel='sigmoid', gamma= 0.01425320509380262, probability=True)

# Define datasets for each comparison in the desired order
datasets = {
    'AZ_vs_HC': (X_train_az_hc, y_train_az_hc, X_test_az_hc, y_test_az_hc),
    'FTD_vs_HC': (X_train_ftd_hc, y_train_ftd_hc, X_test_ftd_hc, y_test_ftd_hc),
    'AZ_vs_FTD': (X_train_ftd_az, y_train_ftd_az, X_test_ftd_az, y_test_ftd_az)
}

# Define the classifiers dictionary in the desired order
classifiers = {
    'AZ_vs_HC': svc_az_hc,
    'FTD_vs_HC': svc_ftd_hc,
    'AZ_vs_FTD': svc_ftd_az
}

# Initialize a dictionary to store the cross-validation results
cv_results = {}

# Loop through each comparison and perform cross-validation
for comparison, clf in classifiers.items():
    X_train, y_train, X_test, y_test = datasets[comparison]

    # Perform cross-validation and store the results
    scores = cross_val_score(clf, X_train, y_train, cv=10)
    cv_results[comparison] = scores

    # Evaluate the SVC model on the test set
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Print results for each model
    print(f"\nSummary of cross-validation results for {comparison}:")
    print(f"{comparison}: Mean accuracy = {scores.mean():.4f}, Standard deviation = {scores.std():.4f}")

    print(f"\nClassification Report for {comparison}:")
    print(classification_report(y_test, y_pred))

    print(f"Confusion Matrix for {comparison}:")
    print(confusion_matrix(y_test, y_pred))

    print(f"Accuracy for {comparison}: {accuracy_score(y_test, y_pred):.4f}")



Summary of cross-validation results for AZ_vs_HC:
AZ_vs_HC: Mean accuracy = 0.8900, Standard deviation = 0.1174

Classification Report for AZ_vs_HC:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           2       0.88      0.88      0.88         8

    accuracy                           0.85        13
   macro avg       0.84      0.84      0.84        13
weighted avg       0.85      0.85      0.85        13

Confusion Matrix for AZ_vs_HC:
[[4 1]
 [1 7]]
Accuracy for AZ_vs_HC: 0.8462

Summary of cross-validation results for FTD_vs_HC:
FTD_vs_HC: Mean accuracy = 0.8550, Standard deviation = 0.1193

Classification Report for FTD_vs_HC:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.80      0.89         5

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.91        11
weighted avg       0.9

Training SVC models for AZ vs HC,FTD vs HC and AZ vs FTD using the best parameters obtained

In [ ]:

# 'SVC': SVC(C=2.1214906873874253,gamma=0.004484292508881285,kernel='rbf',probability=True)
# SVC(C=0.08417824373866711, kernel='linear', probability=True)
svc_az_hc = SVC(C=0.08417824373866711, kernel='linear', probability=True)
svc_az_hc.fit(X_train_az_hc, y_train_az_hc)


#{'C': 218.092808835883, 'kernel': 'sigmoid', 'gamma': 'scale'}
# SVC(C=256.2063292041088, kernel='sigmoid', gamma= 0.01425320509380262, probability=True)
svc_ftd_az = SVC(C=256.2063292041088, kernel='sigmoid', gamma= 0.01425320509380262, probability=True)
svc_ftd_az.fit(X_train_ftd_az, y_train_ftd_az)

# 'SVC'= SVC(C=4.115770269418699,gamma=0.003852086974868314,kernel='poly',probability=True)
#(C= 1.6953850458743818, gamma='scale', kernel='rbf', probability=True)
svc_ftd_hc = SVC(C= 1.6953850458743818, gamma='scale', kernel='rbf', probability=True)
svc_ftd_hc.fit(X_train_ftd_hc, y_train_ftd_hc)

# print(X_train_ftd_hc.shape)
# print(masked_features_patient.shape)




SVC(C=1.6953850458743818, probability=True)

Combined model

In [ ]:
def combined_model(X):
    predictions = []
    for x in X:
        x = x.reshape(1, -1)
        pred_az_hc = svc_az_hc.predict(x)
        pred_ftd_hc = svc_ftd_hc.predict(x)
        if pred_az_hc == 0 and pred_ftd_hc == 0:
            predictions.append(0)  # Healthy Control
        else:
            pred_ftd_az = svc_ftd_az.predict(x)
            if pred_ftd_az == 1:
                predictions.append(1)  # FTD
            else:
                predictions.append(2)  # AZ
    return np.array(predictions)

In [ ]:
# Combine test sets and labels for final evaluation
X_test_combined = np.concatenate((X_test_az_hc, X_test_ftd_hc, X_test_ftd_az), axis=0)
y_test_combined = np.concatenate((y_test_az_hc, y_test_ftd_hc, y_test_ftd_az), axis=0)

# Predict using the combined model
y_pred_combined = combined_model(X_test_combined)

Classification Report for Combined Model

In [ ]:
print("Classification Report for Combined Model:")
print(classification_report(y_test_combined, y_pred_combined))
print("Confusion Matrix for Combined Model:")
print(confusion_matrix(y_test_combined, y_pred_combined))
print("Accuracy for Combined Model:", accuracy_score(y_test_combined, y_pred_combined))

Classification Report for Combined Model:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        11
           1       0.73      0.80      0.76        10
           2       0.86      0.80      0.83        15

    accuracy                           0.83        36
   macro avg       0.83      0.84      0.83        36
weighted avg       0.84      0.83      0.83        36

Confusion Matrix for Combined Model:
[[10  0  1]
 [ 1  8  1]
 [ 0  3 12]]
Accuracy for Combined Model: 0.8333333333333334


In [ ]:


# Predict using the combined model
patient_predictions = combined_model(masked_features_patient)

# Print the predictions
print(patient_predictions)


[1]
